In [28]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

train_url = 'https://storage.googleapis.com/kaggle-competitions-data/kaggle/3136/train.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1528004576&Signature=i3EJD%2FmW%2BVBYKZMLlyhwWg%2Bwv0A6aCeInBC1WYZkKpGNZnhdHUMFwN9NpcRYU9Bk6Rg%2BKzoaqILW5QDI63z8TathqmhgzrWk18O7B3ixLVPHdIqOfaDPyD9dm%2BCFgBP0Iy9FU1B4yWAI0GfymIXQF2fCJjHJZYG4nHOTjmFQwPW1SkBOCyWAe0itNQXBHTv8ETrf7lyiCFvJJZ%2BqDBp7lOkhFua3Fu3lyuUxfLNOdoVMaiNDL5c%2B3pVm7QfSWJyQQylRbeCTfviKDXVt1Qbh3PZ0%2BbQ2S7AENZ4YOGYURkR7z9f5lkRDzLEtqwDdsfT8jqoGtgDcqJbcTTCzq7YpYg%3D%3D'
df = pd.read_csv(train_url)

In [9]:
"""
- 그냥 슬쩍 보기에, 유용하다고 생각되는 것들은 Pclass, Sex, Age 로 나뉨. 
"""
print(df.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [165]:
# 간단히 null 계산

from sklearn.preprocessing import MinMaxScaler
"""
- cabin값이 있고, 없을때의 survival rate가 0.4 정도 차이남.
- 각각, sibling+spouse의 수, parent+child 의 수인데, 개별 class별로 차이가 발생함. 
"""
X = pd.get_dummies(df['Pclass'], prefix='Pclass').join(
    pd.get_dummies(df['Sex'], prefix='Sex')).join(
    df['Age'].fillna(df['Age'].mean())).join(
    df['Fare']).join(
    pd.get_dummies(df['Cabin'].isnull(), prefix='Cab')).join(
    pd.get_dummies(df['SibSp'] + df['Parch'], prefix='fam')).join(
    pd.get_dummies(df['Name'].apply(find_preName)))
X = pd.DataFrame(MinMaxScaler().fit_transform(X), columns=X.columns)
Y = df['Survived']

print(X.columns)

train_X, test_X, train_Y, test_Y = train_test_split(X, Y, train_size=0.8, random_state=42)
"""
prediction 
다양한 모델을 세워서 예측해봅시다. 
"""

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

models = {
    "logisticReg":LogisticRegression(penalty='l1'),
    "DecTreeClf":DecisionTreeClassifier(),
    'neural-net':MLPClassifier(hidden_layer_sizes=[10, 50, 10], activation='relu', solver='adam'),
    'kneighborsClf':KNeighborsClassifier(n_neighbors=10),
    'svc':SVC(kernel='rbf'),#poly, linear, sigmoid
    'randomforest':RandomForestClassifier()
}

for k, m in models.items():
    m.fit(train_X, train_Y)

for k, m in models.items():
    print(k)
    print("train accuracy: {}".format(m.score(train_X, train_Y)))
    print("test accuracy: {}".format(m.score(test_X, test_Y)))
    print("--------")



Index(['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male', 'Age',
       'Fare', 'Cab_False', 'Cab_True', 'fam_0', 'fam_1', 'fam_2', 'fam_3',
       'fam_4', 'fam_5', 'fam_6', 'fam_7', 'fam_10', 'capt', 'col', 'don',
       'dr', 'jonkheer', 'lady', 'major', 'master', 'miss', 'mlle', 'mme',
       'mr', 'mrs', 'ms', 'rev', 'sir', 'the countess'],
      dtype='object')


/Users/frhyme/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


logisticReg
train accuracy: 0.8370786516853933
test accuracy: 0.8212290502793296
--------
DecTreeClf
train accuracy: 0.9859550561797753
test accuracy: 0.7374301675977654
--------
neural-net
train accuracy: 0.8455056179775281
test accuracy: 0.8100558659217877
--------
kneighborsClf
train accuracy: 0.8384831460674157
test accuracy: 0.7932960893854749
--------
svc
train accuracy: 0.7879213483146067
test accuracy: 0.7821229050279329
--------
randomforest
train accuracy: 0.9719101123595506
test accuracy: 0.8100558659217877
--------


In [104]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [114]:
df['Ticket'] #가격, 같은 것일줄 알았는데, 일단은 무슨 의미인지 모르겠음
df['Fare']# 운임요금, 아마도 비싼지 적은지에 따라서 구분이 가능할듯 
df['SibSp']# 이게 가족 뭐였던것 같은데 
df['Parch']# 이것도 가족 뭐 였던 것 같은데 
df[['Cabin', 'Survived']]
df[df['Cabin'].isnull()]['Survived'].mean()

0.29985443959243085

In [131]:
df.groupby('Parch')['Survived'].sum()/df.groupby('Parch')['Survived'].count()

Parch
0    0.343658
1    0.550847
2    0.500000
3    0.600000
4    0.000000
5    0.200000
6    0.000000
Name: Survived, dtype: float64

In [132]:
df.groupby('SibSp')['Survived'].sum()/df.groupby('SibSp')['Survived'].count()

SibSp
0    0.345395
1    0.535885
2    0.464286
3    0.250000
4    0.166667
5    0.000000
8    0.000000
Name: Survived, dtype: float64

In [ ]:
df['t'] = df['SibSp'] + df['Parch']
df.groupby('t')['Survived'].sum()/df.groupby('t')['Survived'].count()
df.groupby('t')['Survived'].count()

In [159]:
def find_preName(inputS):
    inputS = inputS.lower()
    if '.' in inputS:
        return inputS[inputS.find(",")+2:inputS.find(".")] 
    else:
        return "none"
df['preName'] = df['Name'].apply(find_preName)

df.groupby('preName')['Survived'].sum()/df.groupby('preName')['Survived'].count()
#df.groupby('preName')['Survived'].count()

preName
capt            0.000000
col             0.500000
don             0.000000
dr              0.428571
jonkheer        0.000000
lady            1.000000
major           0.500000
master          0.575000
miss            0.697802
mlle            1.000000
mme             1.000000
mr              0.156673
mrs             0.792000
ms              1.000000
rev             0.000000
sir             1.000000
the countess    1.000000
Name: Survived, dtype: float64

In [161]:
df.groupby('preName')['Survived'].count()
"""
master, mrs, miss 의 경우 수도 많고 생존률이 높은 반면 
mr의 경우 생존율이 매우 낮음 
"""

preName
capt              1
col               2
don               1
dr                7
jonkheer          1
lady              1
major             2
master           40
miss            182
mlle              2
mme               1
mr              517
mrs             125
ms                1
rev               6
sir               1
the countess      1
Name: Survived, dtype: int64